In [12]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm
from tqdm import tqdm
from torchvision import transforms
from PIL import Image
import pickle
import numpy as np
import torch.nn.functional as F
from hloc.utils.read_write_model import read_images_binary
import pandas as pd
import os
import json
# Base path configuration (상대 경로)
BASE_PATH = '..'

# Derived paths
PATHS = {
    'model_base': os.path.join(BASE_PATH, 'model'),
    'database': os.path.join(BASE_PATH, 'DataBase'),
    'dataset': os.path.join(BASE_PATH, 'datasets'),
    'global_desc': os.path.join(BASE_PATH, 'GlobalDescriptors'),
    'fold_dataset': os.path.join(BASE_PATH, '5FoldDataset'),
    'outputs': os.path.join(BASE_PATH, 'outputs')
}

TRANSFORM = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

K_VALUES = [200, 150, 100, 50, 20, 5, 1]

# Model configurations
MODEL_CONFIGS = {
    '1st': {'path': os.path.join(PATHS['model_base'], 'B_1_1st_Train_Student/best_model8929_0.6792626728110606.pth')},
    '2nd': {'path': os.path.join(PATHS['model_base'], 'B_1_2nd_Train_Student/best_model9827_0.6837962962962971.pth')},
    '3rd': {'path': os.path.join(PATHS['model_base'], 'B_1_3rd_Train_Student/best_model9887_0.6800925925925929.pth')},
    '4th': {'path': os.path.join(PATHS['model_base'], 'B_1_4th_Train_Student/best_model9244_0.6696759259259274.pth')},
    '5th': {'path': os.path.join(PATHS['model_base'], 'B_1_5th_Train_Student/best_model9397_0.6780092592592604.pth')}
}


In [14]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm
from tqdm import tqdm
from torchvision import transforms
from PIL import Image
import pickle
import numpy as np
import torch.nn.functional as F
from hloc.utils.read_write_model import read_images_binary
import pandas as pd
import os

# Base path configuration (상대 경로)
BASE_PATH = '..'

# Derived paths
PATHS = {
    'model_base': os.path.join(BASE_PATH, 'model'),
    'database': os.path.join(BASE_PATH, 'DataBase'),
    'dataset': os.path.join(BASE_PATH, 'datasets'),
    'global_desc': os.path.join(BASE_PATH, 'GlobalDescriptors'),
    'fold_dataset': os.path.join(BASE_PATH, '5FoldDataset'),
    'outputs': os.path.join(BASE_PATH, 'outputs')
}

TRANSFORM = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

K_VALUES = [200, 150, 100, 50, 20, 5, 1]

# Model configurations
MODEL_CONFIGS = {
    '1st': {'path': os.path.join(PATHS['model_base'], 'B_1_1st_Train_Student/best_model8929_0.6792626728110606.pth')},
    '2nd': {'path': os.path.join(PATHS['model_base'], 'B_1_2nd_Train_Student/best_model9827_0.6837962962962971.pth')},
    '3rd': {'path': os.path.join(PATHS['model_base'], 'B_1_3rd_Train_Student/best_model9887_0.6800925925925929.pth')},
    '4th': {'path': os.path.join(PATHS['model_base'], 'B_1_4th_Train_Student/best_model9244_0.6696759259259274.pth')},
    '5th': {'path': os.path.join(PATHS['model_base'], 'B_1_5th_Train_Student/best_model9397_0.6780092592592604.pth')}
}

class ImageDataset:
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data_frame)
        
    def __getitem__(self, idx):
        image_path = os.path.join(PATHS['dataset'], 'aachen/images/images_upright', self.data_frame['Anchor'][idx])
        anchor_img = Image.open(image_path)
        if self.transform:
            anchor_img = self.transform(anchor_img)
        return anchor_img

def load_model(model_path):
    from models.crossvit_official import crossvit_tiny_224
    model = crossvit_tiny_224(pretrained=True)
    checkpoint = torch.load(model_path)
    new_state_dict = {k[7:] if 'module.' in k else k: v 
                     for k, v in checkpoint['crossvit_state_dict'].items()}
    model.load_state_dict(new_state_dict, strict=True)
    model = model.cuda()
    model = nn.DataParallel(model, device_ids=[0,1])
    return model

def extract_features(model, dataset_path, output_size):
    dataset = ImageDataset(csv_file=dataset_path, transform=TRANSFORM)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=8)
    features = torch.zeros(len(dataset), output_size).cuda()
    
    model.eval()
    with torch.no_grad():
        for i, anchor in enumerate(dataloader):
            anchor = anchor.cuda(non_blocking=True)
            image_encoding = model(anchor)
            image_encoding = F.normalize(image_encoding, p=2, dim=1)
            features[i] = image_encoding
            
    return features.cpu().numpy()

def calculate_neighbors(train_features, query_features, k):
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(train_features)
    distances, indices = nbrs.kneighbors(query_features)
    return distances[:, 1:], indices[:, 1:]

def save_references(indices, query_table, database_table, output_path):
    with open(output_path, "w") as file:
        for i in range(len(indices)):
            for index in indices[i]:
                file.writelines(f"{query_table['Anchor'][i]} {database_table['Anchor'][index]}\n")
    
    # Sort lines
    with open(output_path) as f:
        lines = sorted(f.readlines())
    with open(output_path, 'w') as file:
        for line in lines:
            file.writelines(line)

def calculate_accuracy(reference_file, database_path, images_path):
    images = read_images_binary(images_path)
    with open(database_path, 'rb') as f:
        anc_pos_relations = pickle.load(f)
        
    search_results = {}
    total_image_id = list(images.keys())
    
    with open(reference_file, 'r') as file:
        for line in file:
            anchor_img, result_img = line.strip().split()
            anchor_key = None
            result_key = None
            for j in total_image_id:
                tar_img = images[j]
                if tar_img.name == anchor_img:
                    anchor_key = j
                if tar_img.name == result_img:
                    result_key = j
            if anchor_key not in search_results:
                search_results[anchor_key] = []
            search_results[anchor_key].append(result_key)
            
    total_positive_rates = []
    for anchor_key, results in search_results.items():
        positive_count = 0
        anchor_positives = []
        for item in anc_pos_relations:
            if item['Anchor'][0] == anchor_key:
                anchor_positives = item['Positive']
                break
                
        for result_key in results:
            if result_key in anchor_positives:
                positive_count += 1
                
        denominator = min(len(results), len(anchor_positives))
        positive_rate = positive_count / denominator if denominator > 0 else 0
        total_positive_rates.append(positive_rate)
        
    return np.mean(total_positive_rates)

def process_model(model_name):
    print(f"\nProcessing {model_name} model...")
    
    model = load_model(MODEL_CONFIGS[model_name]['path'])
    
    print("Extracting features...")
    database_path = os.path.join(PATHS['database'], 'DataBase4328.csv')
    query_path = os.path.join(PATHS['fold_dataset'], f'anchor_student_{model_name}.csv')
    
    train_features = extract_features(model, database_path, 1000)
    query_features = extract_features(model, query_path, 1000)
    
    results = {}
    for k in K_VALUES:
        distances, indices = calculate_neighbors(train_features, query_features, k)
        output_path = os.path.join(PATHS['global_desc'], f'{model_name}_StudentReference_{k}.txt')
        
        pd_database = pd.read_csv(database_path)
        query_table = pd.read_csv(query_path)
        
        save_references(indices, query_table, pd_database, output_path)
        accuracy = calculate_accuracy(
            output_path,
            os.path.join(PATHS['database'], 'DataBase_Norm_Tiny.pickle'),
            os.path.join(PATHS['outputs'], 'aachen/sfm_superpoint+superglue/images.bin')
        )
        
        results[k] = accuracy
        print(f"k={k}: {accuracy:.4f}")
    
    return results

def process_all_models():
    all_results = {}
    for model_name in tqdm(MODEL_CONFIGS.keys()):
        results = process_model(model_name)
        all_results[model_name] = results
    return all_results

def display_results(all_results):
    print("\nFinal Results Summary:")
    
    # Print header
    print("\nk values:", end="")
    for k in K_VALUES:
        print(f"\t{k}", end="")
    print()
    
    # Print results for each model
    for model_name in sorted(all_results.keys()):
        print(f"\n{model_name} model:", end="")
        for k in K_VALUES:
            print(f"\t{all_results[model_name][k]:.4f}", end="")
    
    print("\n\n=== Average Values for Each k ===")
    # Calculate and print average for each k separately
    for k in K_VALUES:
        values = [results[k] for results in all_results.values()]
        avg = np.mean(values)
        std = np.std(values)
        max_val = np.max(values)
        min_val = np.min(values)
        max_model = max(all_results.items(), key=lambda x: x[1][k])[0]
        min_model = min(all_results.items(), key=lambda x: x[1][k])[0]
        
        print(f"\nFor k = {k}:")
        print(f"Average: {avg:.4f}")
        print(f"Std Dev: {std:.4f}")
        print(f"Max: {max_val:.4f} (Model: {max_model})")
        print(f"Min: {min_val:.4f} (Model: {min_model})")
        
        # Individual model values
        print("Individual model values:")
        for model_name in sorted(all_results.keys()):
            print(f"  {model_name} model: {all_results[model_name][k]:.4f}")

    # Summary print for just the averages
    print("\n=== Summary of Averages ===")
    print("print('Average values for different k:')")
    for k in K_VALUES:
        avg = np.mean([results[k] for results in all_results.values()])
        print(f"k = {k}: {avg:.4f}")

# Optional: Save detailed results
def save_detailed_results(all_results):
    detailed_results = {
        'individual_results': all_results,
        'statistics': {}
    }
    
    for k in K_VALUES:
        values = [results[k] for results in all_results.values()]
        detailed_results['statistics'][k] = {
            'average': float(np.mean(values)),
            'std_dev': float(np.std(values)),
            'max': {
                'value': float(np.max(values)),
                'model': max(all_results.items(), key=lambda x: x[1][k])[0]
            },
            'min': {
                'value': float(np.min(values)),
                'model': min(all_results.items(), key=lambda x: x[1][k])[0]
            }
        }
    
    with open('precision@K_results.json', 'w') as f:
        json.dump(detailed_results, f, indent=4)

In [15]:
all_results = process_all_models()

  0%|          | 0/5 [00:00<?, ?it/s]


Processing 1st model...
Extracting features...
k=200: 0.8196
k=150: 0.7710
k=100: 0.7177
k=50: 0.6851
k=20: 0.7290
k=5: 0.8363
k=1: 0.9021

Processing 2nd model...
Extracting features...
k=200: 0.8305
k=150: 0.7825
k=100: 0.7303
k=50: 0.6984
k=20: 0.7476
k=5: 0.8432
k=1: 0.8924

Processing 3rd model...
Extracting features...
k=200: 0.8303
k=150: 0.7809
k=100: 0.7259
k=50: 0.6871
k=20: 0.7368
k=5: 0.8426
k=1: 0.9062

Processing 4th model...
Extracting features...
k=200: 0.8157
k=150: 0.7677
k=100: 0.7149
k=50: 0.6847
k=20: 0.7320
k=5: 0.8232
k=1: 0.8843

Processing 5th model...
Extracting features...
k=200: 0.8081
k=150: 0.7615
k=100: 0.7147
k=50: 0.6922
k=20: 0.7475
k=5: 0.8406
k=1: 0.9016


In [16]:
display_results(all_results)


Final Results Summary:

k values:	200	150	100	50	20	5	1

1st model:	0.8196	0.7710	0.7177	0.6851	0.7290	0.8363	0.9021
2nd model:	0.8305	0.7825	0.7303	0.6984	0.7476	0.8432	0.8924
3rd model:	0.8303	0.7809	0.7259	0.6871	0.7368	0.8426	0.9062
4th model:	0.8157	0.7677	0.7149	0.6847	0.7320	0.8232	0.8843
5th model:	0.8081	0.7615	0.7147	0.6922	0.7475	0.8406	0.9016

=== Average Values for Each k ===

For k = 200:
Average: 0.8208
Std Dev: 0.0086
Max: 0.8305 (Model: 2nd)
Min: 0.8081 (Model: 5th)
Individual model values:
  1st model: 0.8196
  2nd model: 0.8305
  3rd model: 0.8303
  4th model: 0.8157
  5th model: 0.8081

For k = 150:
Average: 0.7727
Std Dev: 0.0080
Max: 0.7825 (Model: 2nd)
Min: 0.7615 (Model: 5th)
Individual model values:
  1st model: 0.7710
  2nd model: 0.7825
  3rd model: 0.7809
  4th model: 0.7677
  5th model: 0.7615

For k = 100:
Average: 0.7207
Std Dev: 0.0063
Max: 0.7303 (Model: 2nd)
Min: 0.7147 (Model: 5th)
Individual model values:
  1st model: 0.7177
  2nd model: 0.7303
  3r